This note book contains the scripts for preprocessing financil data from a video, converting them from video to frames and then sending them to VERTEX GEMINI API for extracting data. using offcial Vertex SDK for video uploading.

In [30]:
# # only run if you want to To download video from s3 or any downloadable link convert video into frames and save it into a folder. read the last 2 instruction paras
# import cv2
# import os
# import urllib.request

# def download_video(url, local_path):
#     try:
#         urllib.request.urlretrieve(url, local_path)
#         print(f"Video downloaded from {url} to: {local_path}")
#         return True
#     except Exception as e:
#         print(f"Error downloading video: {e}")
#         return False

# def save_frames_from_video(video_path, output_folder, frame_interval=1):
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     video = cv2.VideoCapture(video_path)
#     if not video.isOpened():
#         print("Error: Could not open video.")
#         return

#     frame_count = 0
#     while True:
#         success, frame = video.read()
#         if not success:
#             break

#         if frame_count % frame_interval == 0:
#             frame_file = os.path.join(output_folder, f"frame_{frame_count}.jpg")
#             cv2.imwrite(frame_file, frame)

#         frame_count += 1

#     video.release()
#     print(f"Total frames saved: {frame_count}")

# # System prompt for video URL
# video_url = input("Enter the URL of the video to download: ")
# local_video_path = "downloaded_video.mp4"  # Local path to save the downloaded video

# # User prompt for frame extraction interval
# frame_interval = int(input("Enter the frame extraction interval (e.g., 1 for every frame, 10 for every 10th frame, etc.): "))

# # Download video from the provided URL
# if download_video(video_url, local_video_path):
#     # Process and save frames from the downloaded video
#     output_directory = "./2Day frames"  # Replace with your desired output folder
#     save_frames_from_video(local_video_path, output_directory)
    
# # # Video URL
# # video_url = "https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/SPX_2D.mp4"
# # local_video_path = "2day Data.mp4"  # Local path to save the downloaded video

# # Download video from the provided URL
# if download_video(video_url, local_video_path):
#     # Process and save frames from the downloaded video
#     output_directory = "./2Day frames"  # Replace with your desired output folder
#     save_frames_from_video(local_video_path, output_directory)


In [3]:
import cv2
import os
import urllib.request

def download_video(url, local_path):
    if os.path.exists(local_path):
        print(f"Video already exists at: {local_path}")
        return True

    try:
        urllib.request.urlretrieve(url, local_path)
        print(f"Video downloaded from {url} to: {local_path}")
        return True
    except Exception as e:
        print(f"Error downloading video: {e}")
        return False

def save_frames_from_video(video_path, output_folder, frame_interval=1):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        print("Error: Could not open video.")
        return

    frame_count = 0
    while True:
        success, frame = video.read()
        if not success:
            break

        if frame_count % frame_interval == 0:
            frame_file = os.path.join(output_folder, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_file, frame)

        frame_count += 1

    video.release()
    print(f"Total frames saved: {frame_count}")

# System prompt for video URL
video_url = input("Enter the URL of the video to download: ")
local_video_path = "downloaded_video.mp4"  # Local path to save the downloaded video

# User prompt for frame extraction interval
frame_interval = int(input("Enter the frame extraction interval (e.g., 1 for every frame, 10 for every 10th frame, etc.): "))

# Download video from the provided URL if not already downloaded
if download_video(video_url, local_video_path):
    # Process and save frames from the downloaded video
    output_directory = "./2Day frames"  # Replace with your desired output folder
    save_frames_from_video(local_video_path, output_directory, frame_interval)


Enter the URL of the video to download: https://0-source-principal-administrative.s3.amazonaws.com/0-public-data/SPX_2D.mp4
Enter the frame extraction interval (e.g., 1 for every frame, 10 for every 10th frame, etc.): 1
Video already exists at: downloaded_video.mp4
Total frames saved: 3499


In [ ]:
!pip install pytesseract

In [ ]:
# import pytesseract
# pytesseract.pytesseract.tesseract_cmd = (
#     r'/usr/bin/tesseract'
# )

In [31]:
# import os
# from PIL import Image
# import pytesseract


# # Path to the folder containing images
# folder_path = '2Day frames'

# # This function uses Tesseract to extract text from a specified region in an image
# def extract_date_from_image(image_path, region):
#     with Image.open(image_path) as img:
#         cropped_img = img.crop(region)
#         text = pytesseract.image_to_string(cropped_img, lang='eng', config='--psm 6')
#         return text.strip()


# # Define the region where the date is located
# date_region = (142, 50, 267, 75)

# # Get all image file paths
# image_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith(('.jpg', '.png'))]

# # Create a dictionary to group images by date
# date_groups = {}

# # Process each image
# for image_path in image_paths:
#     date_text = extract_date_from_image(image_path, date_region)
#     if date_text not in date_groups:
#         date_groups[date_text] = []
#     date_groups[date_text].append(image_path)

# # Now you have a dictionary where the key is the date and the value is a list of image paths with that date
# for date, images in date_groups.items():
#     print(f"Date: {date}, Images: {images}")




In [42]:
#ABOVE CODE WITH 10 MIN TIMEOUT, USE THIS BLOCK IF PROCESSING IS SLOW ABOVE
import os
from PIL import Image
import pytesseract
import time

# Path to the folder containing images
folder_path = '2Day frames'

# This function uses Tesseract to extract text from a specified region in an image
def extract_date_from_image(image_path, region):
    try:
        with Image.open(image_path) as img:
            cropped_img = img.crop(region)
            text = pytesseract.image_to_string(cropped_img, lang='eng', config='--psm 6')
            return text.strip()
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

# Define the region where the date is located
date_region = (142, 50, 267, 75)

# Get all image file paths
image_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith(('.jpg', '.png'))]

# Create a dictionary to group images by date
date_groups = {}

# Set a timeout (in seconds)
timeout = 600  # 10 minutes

# Process each image
start_time = time.time()
for image_path in image_paths:
    elapsed_time = time.time() - start_time
    if elapsed_time > timeout:
        print(f"Timeout reached. Exiting.")
        break

    date_text = extract_date_from_image(image_path, date_region)
    if date_text is not None:
        if date_text not in date_groups:
            date_groups[date_text] = []
        date_groups[date_text].append(image_path)

# Now you have a dictionary where the key is the date and the value is a list of image paths with that date
for date, images in date_groups.items():
    print(f"Date: {date}, Images: {images}")


Date: D: 12/26/22, Images: ['2Day frames\\frame_0.jpg', '2Day frames\\frame_1.jpg', '2Day frames\\frame_10.jpg', '2Day frames\\frame_100.jpg', '2Day frames\\frame_101.jpg', '2Day frames\\frame_102.jpg', '2Day frames\\frame_103.jpg', '2Day frames\\frame_104.jpg', '2Day frames\\frame_105.jpg', '2Day frames\\frame_106.jpg', '2Day frames\\frame_107.jpg', '2Day frames\\frame_108.jpg', '2Day frames\\frame_109.jpg', '2Day frames\\frame_11.jpg', '2Day frames\\frame_110.jpg', '2Day frames\\frame_111.jpg', '2Day frames\\frame_112.jpg', '2Day frames\\frame_113.jpg', '2Day frames\\frame_114.jpg', '2Day frames\\frame_115.jpg', '2Day frames\\frame_116.jpg', '2Day frames\\frame_117.jpg', '2Day frames\\frame_118.jpg', '2Day frames\\frame_119.jpg', '2Day frames\\frame_12.jpg', '2Day frames\\frame_120.jpg', '2Day frames\\frame_121.jpg', '2Day frames\\frame_122.jpg', '2Day frames\\frame_123.jpg', '2Day frames\\frame_124.jpg', '2Day frames\\frame_125.jpg', '2Day frames\\frame_126.jpg', '2Day frames\\frame

In [43]:
new = {}
for date, images in date_groups.items():
    if images:  # Check if there are any images for the date
        new[date] = images[0]  # Add only the first image of the current date

for date, image in new.items():
    print( {date}, {image})


{'D: 12/26/22'} {'2Day frames\\frame_0.jpg'}
{'D: 4/5/23 ¢'} {'2Day frames\\frame_1000.jpg'}
{'D: 4/7/23 ¢'} {'2Day frames\\frame_1015.jpg'}
{'D: 4/11/23'} {'2Day frames\\frame_1038.jpg'}
{'D: 4/13/23'} {'2Day frames\\frame_1067.jpg'}
{'D: 4/17/23'} {'2Day frames\\frame_1109.jpg'}
{'D: 4/19/23'} {'2Day frames\\frame_1144.jpg'}
{'D: 4/21/23'} {'2Day frames\\frame_1164.jpg'}
{'D: 4/25/23'} {'2Day frames\\frame_1191.jpg'}
{'D: 4/27/23'} {'2Day frames\\frame_1221.jpg'}
{'D: 5/1/23 | ¢'} {'2Day frames\\frame_1246.jpg'}
{'D: 5/1/23 ¢'} {'2Day frames\\frame_1248.jpg'}
{'D: 5/3/23 ¢'} {'2Day frames\\frame_1266.jpg'}
{'D: 5/5/23 ¢'} {'2Day frames\\frame_1298.jpg'}
{'D: 5/9/23 ¢'} {'2Day frames\\frame_1318.jpg'}
{'D: 12/28/22'} {'2Day frames\\frame_133.jpg'}
{'D: 5/11/23'} {'2Day frames\\frame_1338.jpg'}
{'D: 5/15/23'} {'2Day frames\\frame_1355.jpg'}
{'D: 5/17/23'} {'2Day frames\\frame_1375.jpg'}
{'D: 5/19/23'} {'2Day frames\\frame_1395.jpg'}
{'D: 5/23/23'} {'2Day frames\\frame_1415.jpg'}
{'D: 5

In [44]:
#only run to make a copy of extracted unique images to new folder
import os
import shutil

# Base directory where new folders will be created
base_directory = '2D extracted frames'  # Replace with your desired path

# Iterate over the dictionary
for date, image_path in new.items():
    # Create a valid directory name for the date
    # Replace invalid characters for folder names (like '/') with an underscore or another valid character
    folder_name = date.replace(':', '_').replace('/', '_')
    directory_path = os.path.join(base_directory)

    # Create the directory if it doesn't exist
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

    # Define the destination path for the image
    destination_path = os.path.join(directory_path, os.path.basename(image_path))

    # Copy the image to the new location
    shutil.copy(image_path, destination_path)

    print(f"Copied {image_path} to {destination_path}")


Copied 2Day frames\frame_0.jpg to 2D extracted frames\frame_0.jpg
Copied 2Day frames\frame_1000.jpg to 2D extracted frames\frame_1000.jpg
Copied 2Day frames\frame_1015.jpg to 2D extracted frames\frame_1015.jpg
Copied 2Day frames\frame_1038.jpg to 2D extracted frames\frame_1038.jpg
Copied 2Day frames\frame_1067.jpg to 2D extracted frames\frame_1067.jpg
Copied 2Day frames\frame_1109.jpg to 2D extracted frames\frame_1109.jpg
Copied 2Day frames\frame_1144.jpg to 2D extracted frames\frame_1144.jpg
Copied 2Day frames\frame_1164.jpg to 2D extracted frames\frame_1164.jpg
Copied 2Day frames\frame_1191.jpg to 2D extracted frames\frame_1191.jpg
Copied 2Day frames\frame_1221.jpg to 2D extracted frames\frame_1221.jpg
Copied 2Day frames\frame_1246.jpg to 2D extracted frames\frame_1246.jpg
Copied 2Day frames\frame_1248.jpg to 2D extracted frames\frame_1248.jpg
Copied 2Day frames\frame_1266.jpg to 2D extracted frames\frame_1266.jpg
Copied 2Day frames\frame_1298.jpg to 2D extracted frames\frame_1298.jp

Copied 2Day frames\frame_564.jpg to 2D extracted frames\frame_564.jpg
Copied 2Day frames\frame_586.jpg to 2D extracted frames\frame_586.jpg
Copied 2Day frames\frame_605.jpg to 2D extracted frames\frame_605.jpg
Copied 2Day frames\frame_630.jpg to 2D extracted frames\frame_630.jpg
Copied 2Day frames\frame_653.jpg to 2D extracted frames\frame_653.jpg
Copied 2Day frames\frame_678.jpg to 2D extracted frames\frame_678.jpg
Copied 2Day frames\frame_704.jpg to 2D extracted frames\frame_704.jpg
Copied 2Day frames\frame_729.jpg to 2D extracted frames\frame_729.jpg
Copied 2Day frames\frame_751.jpg to 2D extracted frames\frame_751.jpg
Copied 2Day frames\frame_770.jpg to 2D extracted frames\frame_770.jpg
Copied 2Day frames\frame_795.jpg to 2D extracted frames\frame_795.jpg
Copied 2Day frames\frame_821.jpg to 2D extracted frames\frame_821.jpg
Copied 2Day frames\frame_844.jpg to 2D extracted frames\frame_844.jpg
Copied 2Day frames\frame_875.jpg to 2D extracted frames\frame_875.jpg
Copied 2Day frames\f

In [52]:
pip install google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


In [4]:
from google.cloud import aiplatform

aiplatform.init(project="crafty-ring-411412")


In [54]:
pip install --upgrade google-cloud-aiplatform

Requirement already up-to-date: google-cloud-aiplatform in c:\users\suyash\appdata\roaming\python\python38\site-packages (1.39.0)
Note: you may need to restart the kernel to use updated packages.


In [55]:
pip install protobuf==3.20

  Using cached protobuf-3.20.0-cp38-cp38-win_amd64.whl (904 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.2



ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

grpcio-status 1.60.0 requires grpcio>=1.60.0, but you'll have grpcio 1.58.0 which is incompatible.
grpcio-status 1.60.0 requires protobuf>=4.21.6, but you'll have protobuf 3.20.0 which is incompatible.
grpc-google-iam-v1 0.13.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you'll have protobuf 3.20.0 which is incompatible.
googleapis-common-protos 1.62.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you'll have protobuf 3.20.0 which is incompatible.
google-cloud-resource-manager 1.11.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0d

    Uninstalling protobuf-4.25.2:
      Successfully uninstalled protobuf-4.25.2


In [10]:
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)

import vertexai
vertexai.init(project="crafty-ring-411412", location="us-central1")

In [11]:
multimodal_model = GenerativeModel("gemini-pro-vision")

In [12]:
#Sorting the frame names and Extracting frames from the folder.
import os
import re
from vertexai.preview.generative_models import Image

def extract_number(file_name):
    # Extract the numeric part from the filename
    match = re.search(r'\d+', file_name)
    return int(match.group()) if match else 0

# Directory containing your images
directory = "./2D extracted frames"

# List all image filenames in the directory
image_filenames = [filename for filename in os.listdir(directory) if filename.endswith(".jpg")]

# Sort the image filenames by their numeric part
sorted_filenames = sorted(image_filenames, key=lambda x: extract_number(x))
print(sorted_filenames)
# Load images in sorted order
images = [Image.load_from_file(os.path.join(directory, filename)) for filename in sorted_filenames]

print(len(images))


['frame_0.jpg', 'frame_133.jpg', 'frame_166.jpg', 'frame_205.jpg', 'frame_229.jpg', 'frame_254.jpg', 'frame_292.jpg', 'frame_321.jpg', 'frame_345.jpg', 'frame_367.jpg', 'frame_389.jpg', 'frame_413.jpg', 'frame_435.jpg', 'frame_464.jpg', 'frame_487.jpg', 'frame_494.jpg', 'frame_504.jpg', 'frame_515.jpg', 'frame_537.jpg', 'frame_543.jpg', 'frame_564.jpg', 'frame_586.jpg', 'frame_605.jpg', 'frame_630.jpg', 'frame_653.jpg', 'frame_678.jpg', 'frame_704.jpg', 'frame_729.jpg', 'frame_751.jpg', 'frame_770.jpg', 'frame_795.jpg', 'frame_821.jpg', 'frame_844.jpg', 'frame_875.jpg', 'frame_899.jpg', 'frame_916.jpg', 'frame_936.jpg', 'frame_955.jpg', 'frame_973.jpg', 'frame_1000.jpg', 'frame_1015.jpg', 'frame_1038.jpg', 'frame_1067.jpg', 'frame_1109.jpg', 'frame_1144.jpg', 'frame_1164.jpg', 'frame_1191.jpg', 'frame_1221.jpg', 'frame_1246.jpg', 'frame_1248.jpg', 'frame_1266.jpg', 'frame_1298.jpg', 'frame_1318.jpg', 'frame_1338.jpg', 'frame_1355.jpg', 'frame_1375.jpg', 'frame_1395.jpg', 'frame_1415.jp

In [4]:
#multiple images per request [block-1]
New = images[:15]
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.4,
    top_k=10,
    candidate_count=1,
    max_output_tokens=2048,
)
generative_multimodal_model = GenerativeModel("gemini-pro-vision")
response = generative_multimodal_model.generate_content([ *New, "Parse values of Date, open, high, low, close from EVERY ONE these images of financial data. provide data in a table."], generation_config=generation_config)
print(response.candidates[0].text)

 | Date         | Open  | High | Low | Close |
|-------------|-------|-------|------|-------|
| 12/22/2022   | 3843.34 | 3846.65 | 3812.22 | 3829.25 |
| 12/23/2022   | 3829.56 | 3858.19 | 3780.78 | 3848.28 |
| 12/27/2022   | 3836.09 | 3895.00 | 3802.34 | 3855.08 |
| 12/28/2022   | 3855.29 | 3874.86 | 3839.14 | 3863.97 |
| 12/29/2022   | 3863.84 | 3903.95 | 3846.67 | 3899.09 |
| 12/30/2022   | 3898.28 | 4015.23 | 3892.59 | 3972.61 |
| 1/3/2023     | 3971.84 | 3972.96 | 3956.14 | 3966.76 |
| 1/4/2023     | 3966.43 | 4038.43 | 3949.06 | 4030.43 |
| 1/5/2023     | 4053.72 | 4092.21 | 4015.55 | 4017.77 |
| 1/6/2023     | 4020.85 | 4148.95 | 4020.44 | 4119.21 |
| 1/9/2023     | 4158.68 | 4195.44 | 4123.36 | 4136.48 |


In [7]:
#multiple images per request [block-2]
New = images[:15]
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.4,
    top_k=10,
    candidate_count=1,
    max_output_tokens=2048,
)
generative_multimodal_model = GenerativeModel("gemini-pro-vision")
response = generative_multimodal_model.generate_content([ *New, "Parse values of Date, open, high, low, close from EVERY ONE these images of financial data. provide data in a table."], generation_config=generation_config)
print(response.candidates[0].text)

 | Date         | Open  | High | Low | Close |
|-------------|-------|-------|------|-------|
| 12/22/2022   | 3843.34 | 3846.65 | 3812.22 | 3829.25 |
| 12/23/2022   | 3829.56 | 3858.19 | 3780.78 | 3848.28 |
| 12/27/2022   | 3836.29 | 3848.46 | 3793.44 | 3835.97 |
| 12/28/2022   | 3855.74 | 3903.95 | 3847.62 | 3899.09 |
| 12/29/2022   | 3910.82 | 4015.39 | 3926.59 | 3972.36 |
| 12/30/2022   | 3972.71 | 4039.21 | 3957.64 | 4016.93 |
| 1/3/2023     | 4053.72 | 4123.36 | 4034.06 | 4060.43 |
| 1/4/2023     | 4085.85 | 4092.41 | 4020.44 | 4027.21 |
| 1/5/2023     | 4020.23 | 4058.68 | 4013.23 | 4036.48 |
| 1/6/2023     | 4095.44 | 4158.44 | 4091.36 | 4136.48 |


In [20]:
#multiple images per request [block-3]
New = images[:15]
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.4,
    top_k=10,
    candidate_count=1,
    max_output_tokens=2048,
)
generative_multimodal_model = GenerativeModel("gemini-pro-vision")
response = generative_multimodal_model.generate_content([ *New, "Parse values of Date, open, high, low, close from EVERY ONE these images of financial data. provide data in a table."], generation_config=generation_config)
print(response.candidates[0].text)

 | Date         | Open  | High | Low | Close |
|-------------|-------|-------|------|-------|
| 12/22/2022   | 3843.34 | 3846.65 | 3812.22 | 3829.25 |
| 12/23/2022   | 3829.56 | 3858.19 | 3780.78 | 3848.28 |
| 12/27/2022   | 3836.29 | 3848.46 | 3793.44 | 3835.97 |
| 12/28/2022   | 3855.74 | 3903.95 | 3847.62 | 3899.09 |
| 12/29/2022   | 3910.82 | 4015.39 | 3926.59 | 3972.36 |
| 12/30/2022   | 3972.71 | 4039.21 | 3957.64 | 4016.93 |
| 1/3/2023     | 4053.72 | 4123.36 | 4034.06 | 4060.43 |
| 1/4/2023     | 4085.85 | 4092.41 | 4020.44 | 4027.21 |
| 1/5/2023     | 4020.23 | 4058.68 | 4013.23 | 4036.48 |
| 1/6/2023     | 4095.44 | 4158.66 | 4080.76 | 4136.84 |


In [21]:
#multiple images per request [block-4]
New = images[:15]
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.4,
    top_k=10,
    candidate_count=1,
    max_output_tokens=2048,
)
generative_multimodal_model = GenerativeModel("gemini-pro-vision")
response = generative_multimodal_model.generate_content([ *New, "Parse values of Date, open, high, low, close from EVERY ONE these images of financial data. provide data in a table."], generation_config=generation_config)
print(response.candidates[0].text)

 | Date         | Open  | High | Low  | Close |
|-------------|-------|-------|-------|-------|
| 12/22/2022   | 3834.36 | 3846.65 | 3812.22 | 3829.25 |
| 12/23/2022   | 3829.56 | 3858.19 | 3780.78 | 3848.28 |
| 12/27/2022   | 3855.29 | 3893.46 | 3794.33 | 3852.97 |
| 12/28/2022   | 3836.08 | 3874.16 | 3802.42 | 3835.08 |
| 12/29/2022   | 3839.74 | 3905.93 | 3824.12 | 3890.09 |
| 12/30/2022   | 3910.82 | 3950.57 | 3897.29 | 3915.25 |
| 1/3/2023     | 3906.6  | 4013.23 | 3896.54 | 3999.09 |
| 1/4/2023     | 3988.28 | 4015.48 | 3926.59 | 3928.86 |
| 1/5/2023     | 3911.84 | 3972.96 | 3885.54 | 3972.61 |
| 1/6/2023     | 3978.14 | 4039.31 | 3957.64 | 4016.95 |
| 1/9/2023     | 4032.71 | 4061.57 | 3949.06 | 4060.43 |
| 1/10/2023    | 4055.72 | 4092.41 | 4015.55 | 4017.77 |
| 1/11/2023    | 4020.85 | 4148.95 | 4020.44 | 4119.21 |
| 1/12/2023    | 4158.68 | 4195.44 | 4123.36 | 4136.48 |


In [22]:
#multiple images per request [block-5]
New = images[:15]
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.4,
    top_k=10,
    candidate_count=1,
    max_output_tokens=2048,
)
generative_multimodal_model = GenerativeModel("gemini-pro-vision")
response = generative_multimodal_model.generate_content([ *New, "Parse values of Date, open, high, low, close from EVERY ONE these images of financial data. provide data in a table."], generation_config=generation_config)
print(response.candidates[0].text)

 | Date         | Open  | High | Low | Close |
|-------------|-------|-------|------|-------|
| 12/22/2022   | 3843.34 | 3846.65 | 3812.22 | 3829.25 |
| 12/23/2022   | 3829.56 | 3858.19 | 3780.78 | 3848.28 |
| 12/27/2022   | 3836.29 | 3848.46 | 3793.44 | 3835.97 |
| 12/28/2022   | 3855.74 | 3903.95 | 3847.62 | 3899.09 |
| 12/29/2022   | 3910.82 | 4015.39 | 3926.59 | 3972.36 |
| 12/30/2022   | 3972.71 | 4039.21 | 3957.64 | 4016.93 |
| 1/3/2023     | 4053.72 | 4123.36 | 4034.06 | 4060.43 |
| 1/4/2023     | 4085.85 | 4092.41 | 4020.44 | 4027.21 |
| 1/5/2023     | 4020.23 | 4058.68 | 4013.23 | 4036.48 |
| 1/6/2023     | 4095.44 | 4158.66 | 4080.76 | 4136.84 |


## Parse the first 15 images 5 times to the model and concatenate all the outputs (only run to check the accuracy of previous unchanged model)

In [4]:
# Initialize an empty list to store the outputs
all_outputs = []

# Execute the code block 5 times
for _ in range(5):
    New = images[:15]
    
    generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.4,
        top_k=10,
        candidate_count=1,
        max_output_tokens=2048,
    )
    
    generative_multimodal_model = GenerativeModel("gemini-pro-vision")
    response = generative_multimodal_model.generate_content([*New, "Parse values of Date, open, high, low, close from EVERY ONE these images of financial data. provide data in a table."], generation_config=generation_config)
    
    # Append the output to the list
    all_outputs.append(response.candidates[0].text)


## Parse the first 15 images 5 times to the model and concatenate all the outputs after prompt engineering

In [13]:
# Initialize an empty list to store the outputs
all_outputs = []

# Execute the code block 5 times
for _ in range(5):
    New = images[:15]
    
    generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.4,
        top_k=10,
        candidate_count=1,
        max_output_tokens=2048,
    )
    
    generative_multimodal_model = GenerativeModel("gemini-pro-vision")
    response = generative_multimodal_model.generate_content([*New, "You are an experienced accountant who does not make any mistakes with numbers. Your task is to parse values of Date, open, high, low, close for the first 15 images of financial data WITHOUT any deviations. I DO NOT want even a number before or after the decimal point to be anything else than the exact actual value you see in the first 15 images. There should be 15 dates and their values of open, high, low and close but there SHOULD NOT be any dates that are not in the first 15 images. I DO NOT want any values or dates that are not in the images Provide data in a json table."], generation_config=generation_config)
    
    # Append the output to the list
    all_outputs.append(response.candidates[0].text)


In [29]:
# !pip install -q google-generativeai

## Concatenating Previous model responses

In [5]:
# Assuming 'all_outputs' is the list containing the outputs from the previous code
# FOR PREVIOUS MODEL CALCULATION ONLY

# Concatenate the responses into a single string
concatenated_output = '\n'.join(all_outputs)

# Now 'concatenated_response' contains the combined text of all 5 responses
print(concatenated_output)



 | Date         | Open  | High | Low | Close |
|-------------|-------|-------|------|-------|
| 12/22/2022   | 3843.34 | 3846.65 | 3812.22 | 3829.25 |
| 12/23/2022   | 3829.56 | 3858.19 | 3780.78 | 3848.28 |
| 12/27/2022   | 3836.29 | 3848.46 | 3793.44 | 3835.97 |
| 12/28/2022   | 3855.74 | 3903.95 | 3847.62 | 3899.09 |
| 12/29/2022   | 3910.82 | 4015.39 | 3926.59 | 3972.36 |
| 12/30/2022   | 3972.71 | 4039.21 | 3957.64 | 4016.93 |
| 1/3/2023     | 4053.72 | 4123.36 | 4034.06 | 4060.43 |
| 1/4/2023     | 4085.85 | 4092.41 | 4020.44 | 4027.21 |
| 1/5/2023     | 4020.23 | 4058.68 | 4013.23 | 4036.48 |
| 1/6/2023     | 4095.44 | 4158.44 | 4091.36 | 4136.48 |
 | Date         | Open  | High | Low  | Close |
|-------------|-------|-------|-------|-------|
| 12/22/2022   | 3834.36 | 3846.65 | 3812.22 | 3829.25 |
| 12/23/2022   | 3829.56 | 3858.19 | 3780.78 | 3848.28 |
| 12/27/2022   | 3855.29 | 3893.46 | 3794.33 | 3852.97 |
| 12/28/2022   | 3836.08 | 3874.16 | 3802.42 | 3835.08 |
| 12/29/2022

## Concatenating New model responses

In [14]:
# Assuming 'all_outputs' is the list containing the outputs from the previous code
# ...

# Concatenate the responses into a single string
concatenated_json = '\n'.join(all_outputs)

# Now 'concatenated_response' contains the combined text of all 5 responses
print(concatenated_json)


 ```json
{
  "data": [
    {
      "Date": "12/22/22",
      "Open": 3834.36,
      "High": 3846.65,
      "Low": 3812.22,
      "Close": 3829.25
    },
    {
      "Date": "12/23/22",
      "Open": 3959.26,
      "High": 3985.19,
      "Low": 3780.78,
      "Close": 3848.28
    },
    {
      "Date": "12/27/22",
      "Open": 3829.02,
      "High": 3836.48,
      "Low": 3794.33,
      "Close": 3795.88
    },
    {
      "Date": "12/28/22",
      "Open": 3755.29,
      "High": 3802.41,
      "Low": 3736.29,
      "Close": 3752.97
    },
    {
      "Date": "12/29/22",
      "Open": 3738.74,
      "High": 3758.16,
      "Low": 3704.22,
      "Close": 3720.38
    },
    {
      "Date": "1/3/23",
      "Open": 3855.29,
      "High": 3878.46,
      "Low": 3794.33,
      "Close": 3852.97
    },
    {
      "Date": "1/4/23",
      "Open": 3839.74,
      "High": 3919.68,
      "Low": 3802.42,
      "Close": 3908.08
    },
    {
      "Date": "1/5/23",
      "Open": 3910.82,
      "High": 3950

## Grouped and averaged data from the 5 responses

### CODE FOR GROUPING AND AVERAGE FOR PREVIOUS MODEL 

In [6]:
#CODE FOR GROUPING AND AVERAGE FOR PREVIOUS MODEL
import pandas as pd
import re
from io import StringIO

# Assuming merged_data_str contains the data as a string
merged_data_str = '\n'.join(all_outputs)

# Clean up the column names by removing non-alphanumeric characters
merged_data_str = re.sub(r'[^a-zA-Z0-9\n ]', '', merged_data_str)

# Convert the string to a DataFrame
merged_data = pd.read_csv(StringIO(merged_data_str), sep=' ', skipinitialspace=True)

# Remove the "Unnamed: 5" column
merged_data = merged_data.loc[:, ~merged_data.columns.str.contains('^Unnamed')]

# Convert the 'Date' column to mm/dd/yyyy format
merged_data['Date'] = pd.to_datetime(merged_data['Date'], format='%m%d%Y', errors='coerce').dt.strftime('%m/%d/%Y')

# Explicitly set the 'Date' column based on the structure of the DataFrame
date_column = 'Date' if 'Date' in merged_data.columns else None

# If 'Date' is not present, adjust the column name accordingly
if date_column is not None:
    # Convert 'Open', 'High', 'Low', 'Close' columns to numeric
    numeric_columns = ['Open', 'High', 'Low', 'Close']
    for col in numeric_columns:
        merged_data[col] = pd.to_numeric(merged_data[col], errors='coerce')
    
    # Adjust decimal point placement
    merged_data[numeric_columns] /= 100
    
    # Group by 'Date' and calculate the mean for 'Open', 'High', 'Low', 'Close'
    merged_data = merged_data.groupby(date_column).mean().reset_index()
    print(merged_data)
else:
    print("'Date' not found in DataFrame. Please adjust the column name.")


          Date      Open         High       Low     Close
0   01/03/2023  2572.120  4049.228000  3963.468  4017.160
1   01/04/2023  4010.938  4050.842000  3968.624  3999.314
2   01/05/2023  3983.572  4031.098000  3962.618  3998.190
3   01/06/2023  4033.602  4108.918000  4021.572  4085.286
4   01/09/2023  4074.700  4106.193333  4007.160  4085.780
5   01/10/2023  4055.720  4092.410000  4015.550  4017.770
6   11/01/2023  4020.850  4148.950000  4020.440  4119.210
7   11/02/2023  4158.680  4195.440000  4123.360  4136.480
8   12/22/2022  3839.748  3846.650000  3812.220  3829.250
9   12/23/2022  3829.560  3858.190000  3780.780  3848.280
10  12/27/2022  3843.850  3875.768000  3795.576  3846.592
11  12/28/2022  3847.786  3886.216000  3827.844  3866.462
12  12/29/2022  3872.992  3949.318000  3869.618  3924.798
13  12/30/2022  3933.068  3998.958000  3920.490  3967.394


### CODE FOR GROUPING AND AVERAGE OF NEW MODEL 

In [15]:
import json
import numpy as np
import re

# Define the group_and_average function with parameters
def group_and_average(data, date_key='Date', open_key='Open', high_key='High', low_key='Low', close_key='Close'):
    grouped_data = {}

    for response in data:
        if isinstance(response, str):
            response = json.loads(response)
            
        for item in response.get('data', []):
            date = item.get(date_key)
            if date in grouped_data:
                grouped_data[date].append(item)
            else:
                grouped_data[date] = [item]

    averaged_data = []

    for date, items in grouped_data.items():
        averages = {
            date_key: date,
            open_key: round(np.mean([float(item[open_key]) for item in items]), 2),
            high_key: round(np.mean([float(item[high_key]) for item in items]), 2),
            low_key: round(np.mean([float(item[low_key]) for item in items]), 2),
            close_key: round(np.mean([float(item[close_key]) for item in items]), 2)
        }
        averaged_data.append(averages)

    return averaged_data

# Assuming concatenated_json_list is a list containing your 5 concatenated JSON responses
# Combine the JSON responses into a single list
concatenated_json_list = [
    json.loads(response) if isinstance(response, str) else response 
    for response in re.findall(r'```json(.+?)```', concatenated_json, re.DOTALL)
]

# Group and average
result = group_and_average(concatenated_json_list)

# Store the result in the 'answer' variable
answer = json.dumps(result, indent=2)

# Print the result
print(answer)


[
  {
    "Date": "12/22/22",
    "Open": 3834.36,
    "High": 3846.65,
    "Low": 3812.22,
    "Close": 3829.25
  },
  {
    "Date": "12/23/22",
    "Open": 3959.26,
    "High": 3985.19,
    "Low": 3780.78,
    "Close": 3848.28
  },
  {
    "Date": "12/27/22",
    "Open": 3829.02,
    "High": 3836.48,
    "Low": 3794.33,
    "Close": 3795.88
  },
  {
    "Date": "12/28/22",
    "Open": 3762.6,
    "High": 3802.67,
    "Low": 3738.7,
    "Close": 3754.71
  },
  {
    "Date": "12/29/22",
    "Open": 3743.36,
    "High": 3760.73,
    "Low": 3708.24,
    "Close": 3724.69
  },
  {
    "Date": "1/3/23",
    "Open": 3855.29,
    "High": 3878.46,
    "Low": 3802.92,
    "Close": 3852.97
  },
  {
    "Date": "1/4/23",
    "Open": 3839.74,
    "High": 3919.68,
    "Low": 3802.42,
    "Close": 3908.08
  },
  {
    "Date": "1/5/23",
    "Open": 3910.82,
    "High": 3963.53,
    "Low": 3887.15,
    "Close": 3935.22
  },
  {
    "Date": "1/6/23",
    "Open": 3963.26,
    "High": 4006.35,
    "Low":

In [18]:
# #one image per repuest
# import time
# for image in New:
#     response = generative_multimodal_model.generate_content([*New,"parse date, open, high, low, close from this image and make a table", image])
#     print(response.candidates[0].text)
#     time.sleep(10)

In [19]:
# #video preprocessing
# import base64

# def convert_video_to_base64(video_file_path):
#     # Read the video file
#     with open(video_file_path, 'rb') as video_file:
#         video_data = video_file.read()

#     # Encode the video data in base64
#     base64_encoded_data = base64.b64encode(video_data)

#     # Convert to a string for easier handling
#     base64_encoded_string = base64_encoded_data.decode('utf-8')

#     return base64_encoded_string

# # Example usage
# video_base64 = convert_video_to_base64('C:/Users/SUYASH/Downloads/downloaded_video.mp4') #replace video location here



In [20]:
# prompt = """
# parse all the readings of date, open, high, low, close present in this video for the forst 15 frames. Provide this data in a table
# """
# generation_config={
#         "max_output_tokens": 2048,
#         "temperature": 0.4,
#         "top_p": 1,
#         "top_k": 32
#     }
# video = Part.from_data(
#     data=video_base64,
#     mime_type="video/mp4",
# )
# contents = [prompt, video]

# responses = multimodal_model.generate_content(contents, stream=True)


# print("\n-------Response--------")
# for response in responses:
#     print(response.text, end="")

## Accuracy for previous unchanged model

## Previous kpi code provided in the doc

In [13]:
# # Creating dataframes from the dictionaries
# true_df = pd.DataFrame(true_data)
# llm_df = pd.DataFrame(llm_data)
# # Since the dates do not match, we will ignore the date comparison and focus on the financial figures
# # We need to ensure that the data is in the same order for comparison, so we'll sort by the date
# true_df['Date'] = pd.to_datetime(true_df['Date'])
# llm_df['Date'] = pd.to_datetime(llm_df['Date'])
# # Aligning LLM response to the true response based on closest date
# llm_aligned_df = pd.DataFrame()
# # For each date in the true data, find the closest date in the LLM data
# for index, row in true_df.iterrows():
#     closest_llm_row = llm_df.iloc[(llm_df['Date'] - row['Date']).abs().argsort()[:1]]
#     llm_aligned_df = llm_aligned_df.append(closest_llm_row)
# # Reset index for proper alignment
# llm_aligned_df = llm_aligned_df.reset_index(drop=True)
# # Calculate the accuracy as 1 - (MAE of LLM response / range of true values)
# # This gives a proportion of correct values within the range
# accuracy = {}
# error_rate = {}
# for column in ['Open', 'High', 'Low', 'Close']:
#     true_range = true_df[column].max() - true_df[column].min()
#     mae = mean_absolute_error(true_df[column], llm_aligned_df[column])
#     accuracy[column] = 1 - (mae / true_range)
#     error_rate[column] = mae / true_range
# accuracy, error_rate

## Code for measuring accurate accuracy based on prompting responses for previous model

In [48]:
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Read the true data from the CSV file
csv_file_path = 'C:/Users/SUYASH/Downloads/processed_1Y2D (1).csv'
true_df = pd.read_csv(csv_file_path)

# Use the 'answer' variable directly (make sure 'answer' is defined before this point)
answer_data = merged_data

# Convert the answer to a DataFrame
llm_df = pd.DataFrame(answer_data)

# Make sure 'Date' column is present in llm_df before converting to datetime
if 'Date' in llm_df.columns:
    llm_df['Date'] = pd.to_datetime(llm_df['Date'])
    llm_df = llm_df.sort_values(by='Date')

    # Convert 'Date' column to datetime format in true_df
    true_df['Date'] = pd.to_datetime(true_df['Date'])
    true_df = true_df.sort_values(by='Date')

     # Initialize the accuracy dictionary
    accuracy = {}

    # Merge DataFrames based on the closest dates
    merged_df = pd.merge_asof(true_df, llm_df, on='Date', direction='nearest')

    # Calculate the accuracy as 1 - (MAE of LLM response / range of true values)
    # This gives a proportion of correct values within the range
    accuracy = {}
    error_rate = {}
    for column in ['Open', 'High', 'Low', 'Close']:
        true_col_x = merged_df[f'{column}_x']
        llm_col_y = merged_df[f'{column}_y']

        mae = mean_absolute_error(true_col_x, llm_col_y)
        true_range = true_col_x.max() - true_col_x.min()

        accuracy[column] = 1 - (mae / true_range)
        error_rate[column] = mae / true_range

    # Display the results
    print(merged_df)
    print("Accuracy:", accuracy)

    overall_accuracy = sum(accuracy.values()) / len(accuracy)
    print("Overall Accuracy:", overall_accuracy)
else:
    print("Error: 'Date' column not found in llm_df.")


          Date   Open_x   High_x    Low_x  Close_x    Open_y       High_y  \
0   2022-12-26  3829.25  3843.34  3846.65  3813.22  3843.850  3875.768000   
1   2022-12-28  3849.28  3829.56  3858.19  3780.78  3847.786  3886.216000   
2   2022-12-30  3839.50  3829.06  3839.85  3800.34  3933.068  3998.958000   
3   2023-01-05  4119.58  4166.79  4186.92  4089.72  3983.572  4031.098000   
4   2023-01-08  4513.39  4578.83  4584.62  4505.75  4074.700  4106.193333   
..         ...      ...      ...      ...      ...       ...          ...   
127 2023-12-10  4327.78  4380.94  4385.85  4311.97  4158.680  4195.440000   
128 2023-12-13  4719.55  4646.20  4738.57  4643.23  4158.680  4195.440000   
129 2023-12-15  4740.56  4714.23  4749.52  4704.69  4158.680  4195.440000   
130 2023-12-19  4698.35  4743.72  4778.01  4697.82  4158.680  4195.440000   
131 2023-12-21  4754.63  4724.29  4772.94  4708.35  4158.680  4195.440000   

        Low_y   Close_y  
0    3795.576  3846.592  
1    3827.844  3866.462

In [53]:
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Assume 'true_df' and 'llm_df' are defined before this point
# ...

# Check if 'Date' column is present in 'llm_df'
if 'Date' in llm_df.columns:
    llm_df['Date'] = pd.to_datetime(llm_df['Date'])
    
    # Check if all dates in 'llm_df' are present in 'true_df' and if 'llm_df' has 15 rows
    matching_dates = llm_df['Date'].isin(true_df['Date'])
    
    if matching_dates.sum() >= 15 and len(llm_df) == 15:
        # Merge DataFrames based on the closest dates
        merged_df = pd.merge_asof(true_df, llm_df, on='Date', direction='nearest')

        # Calculate the accuracy as 1 - (MAE of LLM response / range of true values)
        # This gives a proportion of correct values within the range
        accuracy = {}
        for column in ['Open', 'High', 'Low', 'Close']:
            true_col_x = merged_df[f'{column}_x']
            llm_col_y = merged_df[f'{column}_y']

            mae = mean_absolute_error(true_col_x, llm_col_y)
            true_range = true_col_x.max() - true_col_x.min()

            accuracy[column] = 1 - (mae / true_range)

        # Calculate the dynamic decrease factor based on missing rows and non-matching dates
        missing_rows = 15 - len(llm_df)
        missing_dates = (~matching_dates).sum()
        decrease_factor = max(0.9, 1 - min(missing_rows / 15, missing_dates / 15))

        # Apply the decrease factor to all accuracy values
        for key in accuracy.keys():
            accuracy[key] *= decrease_factor

        # Display the results
        print(merged_df)
        print("Accuracy:", accuracy)

        overall_accuracy = sum(accuracy.values()) / len(accuracy)
        print("Overall Accuracy:", overall_accuracy)

    else:
        # Display the decreased accuracies
        print("Accuracies after decrease:", accuracy)

else:
    print("Error: 'Date' column not found in llm_df.")



Accuracies after decrease: {'Open': 0.36521053267552, 'High': 0.3918412030125672, 'Low': 0.3512691832626599, 'Close': 0.38745308285154334}


## Code for checking new model accuracy with same logic

In [54]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
import json

# Assuming 'answer_data' is a list of dictionaries
answer_data = json.loads(answer)

# Convert the list of dictionaries to a DataFrame
llm_df = pd.DataFrame(answer_data)

# Convert 'Date' column to datetime format
llm_df['Date'] = pd.to_datetime(llm_df['Date'])

# Read the true data from the CSV file
csv_file_path = 'C:/Users/SUYASH/Downloads/processed_1Y2D (1).csv'
true_df = pd.read_csv(csv_file_path)

# Convert 'Date' column to datetime format
true_df['Date'] = pd.to_datetime(true_df['Date'])

# Sort DataFrames based on the 'Date' column
true_df = true_df.sort_values(by='Date')
llm_df = llm_df.sort_values(by='Date')

# Merge DataFrames based on the closest dates
merged_df = pd.merge_asof(true_df, llm_df, on='Date')

# Calculate the accuracy as 1 - (MAE of LLM response / range of true values)
# This gives a proportion of correct values within the range
accuracy = {}
error_rate = {}
for column in ['Open', 'High', 'Low', 'Close']:
    true_range = merged_df[f'{column}_x'].max() - merged_df[f'{column}_x'].min()
    mae = mean_absolute_error(merged_df[f'{column}_x'], merged_df[f'{column}_y'])
    accuracy[column] = 1 - (mae / true_range)
    error_rate[column] = mae / true_range

# Display the results
print(merged_df)
print("Accuracy:", accuracy)

overall_accuracy = sum(accuracy.values()) / len(accuracy)
print("Overall Accuracy:", overall_accuracy)


          Date   Open_x   High_x    Low_x  Close_x   Open_y   High_y    Low_y  \
0   2022-12-26  3829.25  3843.34  3846.65  3813.22  3959.26  3985.19  3780.78   
1   2022-12-28  3849.28  3829.56  3858.19  3780.78  3762.60  3802.67  3738.70   
2   2022-12-30  3839.50  3829.06  3839.85  3800.34  3743.36  3760.73  3708.24   
3   2023-01-05  4119.58  4166.79  4186.92  4089.72  3910.82  3963.53  3887.15   
4   2023-01-08  4513.39  4578.83  4584.62  4505.75  3963.26  4006.35  3932.80   
..         ...      ...      ...      ...      ...      ...      ...      ...   
127 2023-12-10  4327.78  4380.94  4385.85  4311.97  4158.68  4195.44  4123.36   
128 2023-12-13  4719.55  4646.20  4738.57  4643.23  4158.68  4195.44  4123.36   
129 2023-12-15  4740.56  4714.23  4749.52  4704.69  4158.68  4195.44  4123.36   
130 2023-12-19  4698.35  4743.72  4778.01  4697.82  4158.68  4195.44  4123.36   
131 2023-12-21  4754.63  4724.29  4772.94  4708.35  4158.68  4195.44  4123.36   

     Close_y  
0    3848.28

In [20]:
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Assume 'true_df' and 'llm_df' are defined before this point
# ...

# Check if 'Date' column is present in 'llm_df'
if 'Date' in llm_df.columns:
    llm_df['Date'] = pd.to_datetime(llm_df['Date'])
    
    # Check if all dates in 'llm_df' are present in 'true_df' and if 'llm_df' has 15 rows
    matching_dates = llm_df['Date'].isin(true_df['Date'])
    
    if matching_dates.sum() >= 15 and len(llm_df) == 15:
        # Merge DataFrames based on the closest dates
        merged_df = pd.merge_asof(true_df, llm_df, on='Date', direction='nearest')

        # Calculate the accuracy as 1 - (MAE of LLM response / range of true values)
        # This gives a proportion of correct values within the range
        accuracy = {}
        for column in ['Open', 'High', 'Low', 'Close']:
            true_col_x = merged_df[f'{column}_x']
            llm_col_y = merged_df[f'{column}_y']

            mae = mean_absolute_error(true_col_x, llm_col_y)
            true_range = true_col_x.max() - true_col_x.min()

            accuracy[column] = 1 - (mae / true_range)

        # Calculate the dynamic decrease factor based on missing rows and non-matching dates
        missing_rows = 15 - len(llm_df)
        missing_dates = (~matching_dates).sum()
        decrease_factor = max(0.9, 1 - min(missing_rows / 15, missing_dates / 15))

        # Apply the decrease factor to all accuracy values
        for key in accuracy.keys():
            accuracy[key] *= decrease_factor

        # Display the results
        print(merged_df)
        print("Accuracy:", accuracy)

        overall_accuracy = sum(accuracy.values()) / len(accuracy)
        print("Overall Accuracy:", overall_accuracy)

    else:
        # Display the decreased accuracies
        print("Accuracies after decrease:", accuracy)

else:
    print("Error: 'Date' column not found in llm_df.")


Accuracies after decrease: {'Open': 0.7751957918024832, 'High': 0.782185084174348, 'Low': 0.7545517046908067, 'Close': 0.7762201888871573}


In [10]:
answer

'[\n  {\n    "Date": "12/22/22",\n    "Open": 3834.3599999999997,\n    "High": 3846.65,\n    "Low": 3812.3200000000006,\n    "Close": 3829.25\n  },\n  {\n    "Date": "12/23/22",\n    "Open": 3829.56,\n    "High": 3858.19,\n    "Low": 3780.78,\n    "Close": 3848.28\n  },\n  {\n    "Date": "12/27/22",\n    "Open": 3856.631999999999,\n    "High": 3904.7419999999997,\n    "Low": 3850.8,\n    "Close": 3899.8900000000003\n  },\n  {\n    "Date": "12/28/22",\n    "Open": 3908.2160000000003,\n    "High": 3962.9080000000004,\n    "Low": 3901.9600000000005,\n    "Close": 3930.94\n  },\n  {\n    "Date": "12/29/22",\n    "Open": 3935.704,\n    "High": 3998.418,\n    "Low": 3910.1699999999996,\n    "Close": 3979.6099999999997\n  },\n  {\n    "Date": "1/3/23",\n    "Open": 3975.62,\n    "High": 4060.762,\n    "Low": 3957.34,\n    "Close": 4038.238\n  },\n  {\n    "Date": "1/4/23",\n    "Open": 4046.12,\n    "High": 4079.546,\n    "Low": 3990.91,\n    "Close": 4075.41\n  },\n  {\n    "Date": "1/5/23",